In [75]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog

# fetch game data
gamelog = leaguegamelog.LeagueGameLog(season="2024-25")
data = gamelog.get_data_frames()[0]

print(data.head())

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION               TEAM_NAME     GAME_ID  \
0     22024  1610612752               NYK         New York Knicks  0022400061   
1     22024  1610612747               LAL      Los Angeles Lakers  0022400062   
2     22024  1610612750               MIN  Minnesota Timberwolves  0022400062   
3     22024  1610612738               BOS          Boston Celtics  0022400061   
4     22024  1610612748               MIA              Miami Heat  0022400065   

    GAME_DATE      MATCHUP WL  MIN  FGM  ...  DREB  REB  AST  STL  BLK  TOV  \
0  2024-10-22    NYK @ BOS  L  240   43  ...    29   34   20    2    3   12   
1  2024-10-22  LAL vs. MIN  W  240   42  ...    31   46   22    7    8    7   
2  2024-10-22    MIN @ LAL  L  240   35  ...    35   47   17    4    1   16   
3  2024-10-22  BOS vs. NYK  W  240   48  ...    29   40   33    6    3    4   
4  2024-10-23  MIA vs. ORL  L  240   32  ...    32   41   24    5    8   11   

   PF  PTS  PLUS_MINUS  VIDEO_AVAILABL

In [76]:
# split data into home and away teams
home_teams = data[data['MATCHUP'].str.contains("vs.")]
away_teams = data[data['MATCHUP'].str.contains("@")]

# merge data for the same game based on GAME_ID
combined = pd.merge(home_teams, away_teams, on='GAME_ID', suffixes=('_HOME', '_AWAY'))

# select relevant columns for analysis
# keep stats for home and away teams, and any additional metadata
combined = combined[[
    'GAME_ID',
    'TEAM_ID_HOME', 'PTS_HOME', 'REB_HOME', 'AST_HOME', 'STL_HOME', 'BLK_HOME', 'TOV_HOME',
    'TEAM_ID_AWAY', 'PTS_AWAY', 'REB_AWAY', 'AST_AWAY', 'STL_AWAY', 'BLK_AWAY', 'TOV_AWAY',
    'WL_HOME'
]]

# rename the columns for clarity
combined = combined.rename(columns={'WL_HOME': 'HOME_WIN'})

# convert WL_home to a binary outcome: 1 if home team won, 0 otherwise
combined['HOME_WIN'] = combined['HOME_WIN'].apply(lambda x: 1 if x == 'W' else 0)

# save or analyze the reshaped data
print(combined.head())

      GAME_ID  TEAM_ID_HOME  PTS_HOME  REB_HOME  AST_HOME  STL_HOME  BLK_HOME  \
0  0022400062    1610612747       110        46        22         7         8   
1  0022400061    1610612738       132        40        33         6         3   
2  0022400065    1610612748        97        41        24         5         8   
3  0022400072    1610612757       104        42        21        10         4   
4  0022400071    1610612746       113        51        27         9         1   

   TOV_HOME  TEAM_ID_AWAY  PTS_AWAY  REB_AWAY  AST_AWAY  STL_AWAY  BLK_AWAY  \
0         7    1610612750       103        47        17         4         1   
1         4    1610612752       109        34        20         2         3   
2        11    1610612753       116        57        28         8         8   
3        18    1610612744       140        57        38        13         5   
4        22    1610612756       116        42        25        12         9   

   TOV_AWAY  HOME_WIN  
0        16   

In [74]:
combined.to_csv("../data/game_data.csv", index=False)